# 2-Table Assembling

This notebook assembles the results of the [scraping](https://github.com/njparker1993/oscars_predictions/blob/master/scraping.ipynb) into one dataframe into a machine learning-ready DataFrame.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#nominated and winners of best picture award Oscars from 1927 (year, title, wiki, winner(T/F))
osc_scrape = pd.read_csv('./data/scraping_results/osc_bp.csv')
#oscar nominations of award winning films (year, film, link, liczba nominacji, film_text)
noms = pd.read_csv('./data/scraping_results/noms.csv')
#like osc_scrape but for golden globes from 1943
gg_drama = pd.read_csv('./data/scraping_results/gg_drama.csv')
#like osc_scrape but for golden globes from 1951
gg_comedy = pd.read_csv('./data/scraping_results/gg_comedy.csv')
#producers guild year film wiki winner from 1989
pga = pd.read_csv('./data/scraping_results/pga.csv')
#from 1960
bafta = pd.read_csv('./data/scraping_results/bafta.csv')
#directors guild from 1948
dga = pd.read_csv('./data/scraping_results/dgas.csv')
#screen actors guild from 1995
sag = pd.read_csv('./data/scraping_results/sag_ensemble.csv')
# cannes from 1946
cannes = pd.read_csv('./data/scraping_results/cannes.csv')

In [3]:
cannes

,year,film,wiki,winner
0,1946,Brief Encounter,/wiki/Brief_Encounter,True
1,1946,The Last Chance (1945 film),/wiki/The_Last_Chance_(1945_film),True
2,1946,The Lost Weekend (film),/wiki/The_Lost_Weekend_(film),True
3,1946,María Candelaria,/wiki/Mar%C3%ADa_Candelaria,True
4,1946,Men Without Wings,/wiki/Men_Without_Wings,True
...,...,...,...,...
97,2018,Shoplifters (film),/wiki/Shoplifters_(film),True
98,2019,Parasite (2019 film),/wiki/Parasite_(2019_film),True
99,2020,COVID-19 pandemic,/wiki/COVID-19_pandemic,True
100,2021,Titane (film),/wiki/Titane_(film),True


# Join on Nominations

In [4]:
# I use this order and manually fill in the nominations for this year
noms_merge = noms[['film','nominations']].drop_duplicates()
osc_scrape = pd.merge(osc_scrape, noms_merge, on = 'film', how = 'left')
osc_to_avg = osc_scrape.copy()
avg_noms = int(osc_to_avg.loc[(~osc_to_avg['nominations'].isna())]['nominations'].mean())
osc_scrape.loc[(osc_scrape['nominations'].isna()), 'nominations'] = avg_noms

## Join on the other awards shows

In [5]:
def table_assemble(main_df, to_add_df, show_name):
    """
    Given an awards show scraped from Wikipedia, this function
    Adds it as a feature as a 0/1 flag if it was
    Nominated for that Award and if it won
    """
    nom_col = str('nom_') + show_name
    win_col = str('winner_') + show_name
    # Initalize Columns as no
    main_df[nom_col] = 0
    main_df[win_col] = 0
    to_add_df.columns = ['year','film','wiki','winner_add']
    main_df = pd.merge(main_df, to_add_df[['film','winner_add']], on = 'film', how = 'left')
    main_df.loc[(~main_df['winner_add'].isna()), nom_col] = 1
    main_df.loc[(main_df['winner_add'] == True), win_col] = 1
    main_df = main_df.drop('winner_add', axis = 1)
    return main_df


In [6]:
# Adding a response columns
osc_scrape['Oscar_win'] = 0
osc_scrape.loc[(osc_scrape['winner'] == True), 'Oscar_win'] = 1

In [7]:
# Adding on the results of each festival
scraped_dfs = [gg_drama, gg_comedy, pga, bafta, dga, sag, cannes]
scraped_names = ['gg_drama','gg_comedy','pga', 'bafta', 'dga', 'sag', 'cannes']

for i in range(len(scraped_dfs)):
    osc_scrape = table_assemble(osc_scrape, scraped_dfs[i], scraped_names[i])

In [8]:
# Some awards didn't exist until certain years
for i in range(len(scraped_dfs)):
    print(scraped_names[i], scraped_dfs[i].year.min())

gg_drama 1943
gg_comedy 1951
pga 1989
bafta 1960
dga 1948
sag 1995
cannes 1946


In [9]:
# Saving the data
osc_scrape.to_csv('./data/processed_results/osc_df')

# Results
The DataFrame is now Machine Learning ready, with 0's and 1's as markers for how a given Oscar-Nominated Film did for that Awards show. The response column will be Oscar_win. An example of this final DataFrame can be seen below.

In [17]:
cols = list(osc_scrape.columns)
display_cols = cols[0:2] + cols[4:]
osc_scrape[display_cols].sample(5)

,year,film,nominations,Oscar_win,nom_gg_drama,winner_gg_drama,nom_gg_comedy,winner_gg_comedy,nom_pga,winner_pga,nom_bafta,winner_bafta,nom_dga,winner_dga,nom_sag,winner_sag,nom_cannes,winner_cannes
410,1998,Shakespeare in Love,13.0,1,0,0,1,1,1,0,1,1,1,0,1,1,0,0
24,1931,Shanghai Express,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
413,1998,Saving Private Ryan,11.0,0,1,1,0,0,1,1,1,0,1,1,1,0,0,0
120,1942,Mrs. Miniver,12.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
108,1940,Our Town,7.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
len(osc_scrape)

591

In [18]:
display_cols

['year',
 'film',
 'nominations',
 'Oscar_win',
 'nom_gg_drama',
 'winner_gg_drama',
 'nom_gg_comedy',
 'winner_gg_comedy',
 'nom_pga',
 'winner_pga',
 'nom_bafta',
 'winner_bafta',
 'nom_dga',
 'winner_dga',
 'nom_sag',
 'winner_sag',
 'nom_cannes',
 'winner_cannes']